In [93]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

usedu_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

usedu_df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [68]:
usedu_df1 = usedu_df.copy()
for col in usedu_df1:
    usedu_df1.loc[:, col] = usedu_df1.loc[:, col].interpolate()

usedu_df1.dropna(inplace=True)
usedu_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 36 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1456 non-null object
STATE                           1456 non-null object
YEAR                            1456 non-null int64
ENROLL                          1456 non-null float64
TOTAL_REVENUE                   1456 non-null float64
FEDERAL_REVENUE                 1456 non-null float64
STATE_REVENUE                   1456 non-null float64
LOCAL_REVENUE                   1456 non-null float64
TOTAL_EXPENDITURE               1456 non-null float64
INSTRUCTION_EXPENDITURE         1456 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1456 non-null float64
OTHER_EXPENDITURE               1456 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1456 non-null float64
GRADES_PK_G                     1456 non-null float64
GRADES_KG_G                     1456 non-null float64
GRADES_4_G                      1456 non-null float64
GRADES_8_G                      

1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [76]:
usedu_df1["WEIGHTED_AVG_SCORE"] = (usedu_df1["GRADES_4_G"]*((usedu_df1["AVG_MATH_4_SCORE"] + usedu_df1["AVG_READING_4_SCORE"])*0.5) + usedu_df1["GRADES_8_G"]
                                 * ((usedu_df1["AVG_MATH_8_SCORE"] + usedu_df1["AVG_READING_8_SCORE"])*0.5))/(usedu_df1["GRADES_4_G"] + usedu_df1["GRADES_8_G"])




usedu_df1[["WEIGHTED_AVG_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE", "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", 
          "GRADES_4_G", "GRADES_8_G"]].head()
          



,WEIGHTED_AVG_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,GRADES_4_G,GRADES_8_G
36,243.700100,221.250787,261.870243,219.515870,268.128094,6587.0,7211.0
37,241.266487,219.256638,261.861030,220.319806,268.131990,48793.0,44163.0
38,242.275252,217.262490,261.851817,222.312769,269.755214,41443.0,39610.0
39,242.748345,215.268341,261.842604,224.305732,271.378439,131248.0,126293.0
40,239.781374,219.940098,261.833392,215.449248,265.907109,11129.0,10204.0


2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [77]:
usedu_df1[["WEIGHTED_AVG_SCORE", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,WEIGHTED_AVG_SCORE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
WEIGHTED_AVG_SCORE,1.000000,0.204239,0.207131,0.221824,0.169775,0.128087
TOTAL_EXPENDITURE,0.204239,1.000000,0.992698,0.992435,0.951726,0.928129
INSTRUCTION_EXPENDITURE,0.207131,0.992698,1.000000,0.979165,0.920297,0.895527
SUPPORT_SERVICES_EXPENDITURE,0.221824,0.992435,0.979165,1.000000,0.953411,0.905265
OTHER_EXPENDITURE,0.169775,0.951726,0.920297,0.953411,1.000000,0.923468
CAPITAL_OUTLAY_EXPENDITURE,0.128087,0.928129,0.895527,0.905265,0.923468,1.000000


INSTRUCTION_EXPENDITURE is the most correlated with the new variable WEIGHTED_AVG_SCORE

3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [94]:
X = usedu_df1[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
usedu_df1["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by the',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by the component from Sklearn PCA.
 [0.94725496]


4. What is the correlation between the overall score variable and the 1st principal component?

In [96]:
usedu_df1[["WEIGHTED_AVG_SCORE", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,WEIGHTED_AVG_SCORE,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
WEIGHTED_AVG_SCORE,1.000000,0.187067,0.204239,0.207131,0.221824,0.169775,0.128087
pca_1,0.187067,1.000000,0.992988,0.975096,0.986139,0.975451,0.956156
TOTAL_EXPENDITURE,0.204239,0.992988,1.000000,0.992698,0.992435,0.951726,0.928129
INSTRUCTION_EXPENDITURE,0.207131,0.975096,0.992698,1.000000,0.979165,0.920297,0.895527
SUPPORT_SERVICES_EXPENDITURE,0.221824,0.986139,0.992435,0.979165,1.000000,0.953411,0.905265
OTHER_EXPENDITURE,0.169775,0.975451,0.951726,0.920297,0.953411,1.000000,0.923468
CAPITAL_OUTLAY_EXPENDITURE,0.128087,0.956156,0.928129,0.895527,0.905265,0.923468,1.000000


The correlation between the overall score variable and the 1st principal component is 0.187067

5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

No, I would not choose the 1st principle component instead of the expensiture values because in this case the correlation between the overall score variable and the 1st principal component is (0.187067) is lower than the highest value of correlation of the expenditure variables which is belong to the instruction expenditure and overall score (0.207131).